In [1]:
# Boulder Opal
from qctrl import Qctrl

qctrl = Qctrl()

import time

import jsonpickle
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
from qctrlopencontrols import new_primitive_control
from qctrlvisualizer import QCTRL_STYLE_COLORS, plot_controls, get_qctrl_style

In [2]:
plt.style.use(get_qctrl_style())

# physical constants
h_bar = 1.054571817e-34
c_light = 3e8
k_b = 1.380649e-23

# Rb atom parameters
m_Rb = 86.9092 * 1.6605e-27
lamda = 780 * 1e-9  # m
k_avr = 2 * np.pi / lamda


# three-level matrices for the Raman Hamiltonian
identity3 = np.eye(3, dtype=complex)
sigma_m13 = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [1.0, 0.0, 0.0]], dtype=complex)
sigma_m23 = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 1.0, 0.0]], dtype=complex)

# driving parameters
omega_eff = 2 * np.pi * 4.0 * 10e3  # Hz

# detunings parameters
small_delta = 0  # two-photon
big_delta = 10e9  # one-photon

# dicts used to hold controls for M and BS pulses
pi_half_pulse_controls = {}
pi_pulse_controls = {}

In [3]:
# if True, longer optimization & simulation results will be loaded from file
read_flag = True
# files to save to or load from
data_dir = "./resources/boosting-signal-to-noise-by-10x-in-cold-atom-sensors-using-robust-control/"
Optimized_BS_pulse_file = data_dir + "AtomSensing_Optimized_BS_pulse"
Optimized_M_pulse_file = data_dir + "AtomSensing_Optimized_M_pulse"
Fringe_signals_file = data_dir + "AtomSensing_Fringe_signals"


In [4]:
# helper functions
def run_optimization(duration, target_operator):
    """
    This function specifies the control drive variables
    and the cost function for the optimizer.
    It outputs the final cost and the optimized controls.
    """
    number_of_segments = 256  # number of pulse segments
    number_of_optimization_variables = 128
    omega_max = omega_eff / np.sqrt(2)  # maximum effective Rabi rate
    cutoff_frequency = 4 * omega_eff
    shift_max = 5 * omega_eff  # p/m limit on the freq. shift control

    # define Pauli matrices for 2 level optimization
    sigma_z = np.array([[1.0, 0.0], [0.0, -1.0]], dtype=complex)
    sigma_x = np.array([[0.0, 1.0], [1.0, 0.0]], dtype=complex)
    sigma_y = np.array([[0.0, -1.0j], [1.0j, 0.0]], dtype=complex)

    # define parameters of the stochastic optimization
    sample_count = 300

    # create optimization graph
    graph = qctrl.create_graph()

    # sample random variables from probability distribution
    relative_deltas = graph.random_normal(
        mean=0, standard_deviation=1.0, shape=(sample_count,)
    )
    betas = graph.random_normal(mean=0, standard_deviation=0.4, shape=(sample_count,))

    # set up for drive controls

    # create I & Q variables
    I_values = graph.optimization_variable(
        count=number_of_optimization_variables,
        lower_bound=-omega_max,
        upper_bound=omega_max,
    )
    Q_values = graph.optimization_variable(
        count=number_of_optimization_variables,
        lower_bound=-omega_max,
        upper_bound=omega_max,
    )

    # create sinc filter to smooth and bandlimit the pulses (if desired)
    sinc_kernel = graph.sinc_convolution_kernel(cutoff_frequency)

    # create I & Q signals and apply sinc filter
    I_signal_filtered = graph.convolve_pwc(
        pwc=graph.pwc_signal(values=I_values, duration=duration), kernel=sinc_kernel
    )
    Q_signal_filtered = graph.convolve_pwc(
        pwc=graph.pwc_signal(values=Q_values, duration=duration), kernel=sinc_kernel
    )

    # re-discretize signal
    I_signal = graph.discretize_stf(
        stf=I_signal_filtered,
        duration=duration,
        segment_count=number_of_segments,
        name="i",
    )
    Q_signal = graph.discretize_stf(
        stf=Q_signal_filtered,
        duration=duration,
        segment_count=number_of_segments,
        name="q",
    )

    # set up drive signal for export
    drive_signal = I_signal + 1j * Q_signal
    drive_signal.name = "$\omega$"

    # set up for shift controls

    # create frequency shift variables
    S_values = graph.optimization_variable(
        count=number_of_optimization_variables,
        lower_bound=-shift_max,
        upper_bound=shift_max,
    )

    # create S signals and apply sinc filter
    S_signal_filtered = graph.convolve_pwc(
        pwc=graph.pwc_signal(values=S_values, duration=duration), kernel=sinc_kernel
    )

    # re-discretize signal
    S_signal = graph.discretize_stf(
        stf=S_signal_filtered,
        duration=duration,
        segment_count=number_of_segments,
        name="$\zeta$",
    )

    # set up multiple quasi-static Hamiltonian terms

    # shift term (constant for all points)
    shift_term = S_signal * sigma_z / 2.0

    # create batch of delta terms
    delta_term = graph.constant_pwc_operator(
        duration=duration,
        operator=relative_deltas[:, None, None]
        * (-h_bar * k_avr ** 2 / m_Rb)
        * sigma_z
        / 2.0,
    )

    # create Hamiltonian drive terms
    I_term = I_signal * sigma_x / 2.0
    Q_term = Q_signal * sigma_y / 2.0

    # combine into net drive term (with a batch of noises)
    beta_pwc = graph.constant_pwc(
        constant=betas, duration=duration, batch_dimension_count=1
    )
    drive_term = (1 + beta_pwc) * (I_term + Q_term)

    # total batch of Hamiltonians
    quasistatic_Hamiltonian = drive_term + shift_term + delta_term

    # calculate sum of infidelities for all noise realizations
    infidelity_batch = graph.infidelity_pwc(
        hamiltonian=quasistatic_Hamiltonian, target=graph.target(target_operator)
    )
    cost = graph.sum(infidelity_batch, name="cost")

    # run optimization
    return qctrl.functions.calculate_stochastic_optimization(
        graph=graph,
        cost_node_name="cost",
        output_node_names=["$\omega$", "$\zeta$"],
        iteration_count=10000,
    )

In [5]:

def three_level_controls_from_Open_pulse(pulse):
    """
    This function maps Open Controls pulses designed for a two-level system
    onto the three-level Raman Hamiltonian,
    achieving equivalent evolution of the two ground states.
    """
    # move from effective Rabi rate to two equal laser drives rates
    omega_1 = np.sqrt(pulse.rabi_rates / 2 * big_delta)
    omega_2 = pulse.rabi_rates / 2 * big_delta / omega_1

    # assign half of effective phase to each drive
    phasors_1 = omega_1 * np.exp(1j * (pulse.azimuthal_angles / 2))
    phasors_2 = omega_2 * np.exp(-1j * (pulse.azimuthal_angles / 2))

    # place pulse segments into the required dictionary control format for qctrlcore

    controls = {
        "duration": np.sum(pulse.durations),
        "Omega_1": {"durations": pulse.durations, "values": phasors_1},
        "Omega_2": {"durations": pulse.durations, "values": phasors_2},
    }

    return controls

In [6]:

def three_level_controls_from_two_level_optimized_result(optimized_result):
    """
    This function converts optimized controls for the effective two-level system
    to the three-level Raman system.
    """
    controls = {}

    # drive for a 2 level system translates onto sigma_12 and sigma_23 drives
    if "$\omega$" in optimized_result:

        effective_phasors = np.array(
            [segment["value"] for segment in optimized_result["$\omega$"]]
        )
        durations = np.array(
            [segment["duration"] for segment in optimized_result["$\omega$"]]
        )

        omega_1 = np.sqrt(np.abs(effective_phasors) / 2 * big_delta)
        omega_2 = np.abs(effective_phasors) / 2 * big_delta / omega_1

        # assign half of effective phase to each drive
        phases = np.angle(effective_phasors)
        phasors_1 = omega_1 * np.exp(-1j * phases / 2)
        phasors_2 = omega_2 * np.exp(+1j * phases / 2)

        controls["duration"] = np.sum(durations)
        controls["Omega_1"] = {"durations": durations, "values": phasors_1}
        controls["Omega_2"] = {"durations": durations, "values": phasors_2}

    # shift only for a 2 level system translated onto sigma_12 and sigma_23 drives
    if "$\zeta$" in optimized_result:

        S_values = np.array(
            [segment["value"] for segment in optimized_result["$\zeta$"]]
        )
        shift_durations = np.array(
            [segment["duration"] for segment in optimized_result["$\zeta$"]]
        )

        controls["zeta"] = {"durations": shift_durations, "values": S_values}

    return controls


In [7]:
def calculate_unitary(controls, phase, beta, big_delta, small_delta):
    """
    This function calculates the unitary evolution operator in the Raman system from a
    set of pulse controls, and system parameters (phase offset, amplitude noise beta,
    and momentum/detuning noises small_delta/big_delta).

    The input parameters (phase, beta, big_delta, and small_delta) can be either scalars
    or (compatibly-shaped) batches. In that case, a batch of unitaries is returned,
    one for each set of parameters in the input batch.
    """

    graph = qctrl.create_graph()

    phase = np.array(phase)

    hamiltonian = 0.0

    if "Omega_1" in controls:
        values = (
            (1 + beta[..., None])
            * graph.exp(1j * phase[..., None] / 2)
            * controls["Omega_1"]["values"]
        )
        hamiltonian += 2.0 * graph.pwc_operator_hermitian_part(
            graph.pwc(
                values=values,
                durations=controls["Omega_1"]["durations"],
                time_dimension=-1,
            )
            * sigma_m13
        )

    if "Omega_2" in controls:
        values = (
            (1 + beta[..., None])
            * graph.exp(-1j * phase[..., None] / 2)
            * controls["Omega_2"]["values"]
        )
        hamiltonian += 2.0 * graph.pwc_operator_hermitian_part(
            graph.pwc(
                values=values,
                durations=controls["Omega_2"]["durations"],
                time_dimension=-1,
            )
            * sigma_m23
        )

    if "zeta" in controls:
        hamiltonian += graph.pwc(
            values=controls["zeta"]["values"], durations=controls["zeta"]["durations"]
        ) * (np.diag([-1.0, 1.0, 0.0]) / 2.0)

    drift = graph.constant_pwc(
        constant=big_delta[..., None, None] * np.diag([-1, -1, 0])
        + small_delta[..., None, None] * np.diag([0, -1, 0]),
        duration=controls["duration"],
        batch_dimension_count=len(beta.shape),
    )

    hamiltonian += drift

    unitaries = graph.time_evolution_operators_pwc(
        hamiltonian=hamiltonian,
        sample_times=np.array([controls["duration"]]),
        name="unitaries",
    )

    result = qctrl.functions.calculate_graph(
        graph=graph, output_node_names=["unitaries"]
    )

    unitaries = result.output["unitaries"]["value"]

    return unitaries


In [8]:

def calculate_infidelities(controls, phase, beta, big_delta, small_delta, target):
    """
    This function calculates the infidelity with respect to some target operator
    of the evolution of a the Raman system, from a set of pulse controls and
    system parameters (phase offset, amplitude noise beta, and momentum/detuning
    noises small_delta/big_delta).

    The input parameters (phase, beta, big_delta, and small_delta) can be either scalars
    or (compatibly-shaped) batches. In that case, a batch of infidelities is returned,
    one for each set of parameters in the input batch.
    """

    unitaries = calculate_unitary(
        controls=controls,
        phase=phase,
        beta=beta,
        big_delta=big_delta,
        small_delta=small_delta,
    )

    trace = lambda mat: np.trace(mat, axis1=-1, axis2=-2)
    herm = lambda mat: np.conj(np.transpose(mat))

    target_op = np.matmul(target, np.diag([1, 1, 0]))
    overlap = trace(herm(target_op) @ unitaries) / trace(herm(target_op) @ target_op)
    infidelity = (1 - np.abs(overlap) ** 2).squeeze()

    return infidelity

In [9]:

def pulse_1D_quasi_static_scan(controls, target, betas, del_pz_coefficients):
    """
    This function performs the quasi-static scan against pairs of beta and del_pz_coefficients.
    betas, del_pz_coefficients arrays need to be of the same length.
    """

    big_deltas = np.array(big_delta + del_pz_coefficients * k_avr / m_Rb)
    small_deltas = np.array(-2 * del_pz_coefficients * k_avr / m_Rb)
    betas = np.array(betas)

    return calculate_infidelities(
        controls=controls,
        phase=0.0,
        beta=betas,
        big_delta=big_deltas,
        small_delta=small_deltas,
        target=target,
    )


def pulse_2D_quasi_static_scan(controls, target, betas, del_pz_coefficients):
    """
    This function computes the 2D quasi-static scan
    for each combination of betas and del_pz_coefficients.
    """

    pz_axis, beta_axis = np.meshgrid(del_pz_coefficients, betas)

    big_deltas = np.array(big_delta + pz_axis * k_avr / m_Rb)
    small_deltas = np.array(-2 * pz_axis * k_avr / m_Rb)

    infidelities = calculate_infidelities(
        controls=controls,
        phase=0.0,
        beta=beta_axis,
        big_delta=big_deltas,
        small_delta=small_deltas,
        target=target,
    )

    return pz_axis, beta_axis, infidelities

In [10]:
# JSON based r/w helper functions, type independent
def save_var(file_name, var):
    # Save a single var to a file using json
    f = open(file_name, "w+")
    to_write = jsonpickle.encode(var)
    f.write(to_write)
    f.close()


def load_var(file_name):
    # Load a var from a json file
    f = open(file_name, "r+")
    encoded = f.read()
    decoded = jsonpickle.decode(encoded)
    f.close()

    return decoded

In [11]:
optimization_scheme = "Q-CTRL"

# duration of one effective Rabi cycle period
s = 2 * np.pi / omega_eff

# BS pulse
pi_half_duration = 8 * s  # duration of the BS pulse
pi_half_target_operator = np.array(
    [[np.sqrt(2) / 2, -1j * np.sqrt(2) / 2], [-1j * np.sqrt(2) / 2, np.sqrt(2) / 2]],
    dtype=complex,
)
if not read_flag:
    print("**Optimizing Beam-Splitter pulse**")
    start_time = time.time()
    # the optimization routine
    results = run_optimization(pi_half_duration, pi_half_target_operator)
    save_var(Optimized_BS_pulse_file, results)
    print("run (in min):", (time.time() - start_time) / 60.0)
else:
    results = load_var(Optimized_BS_pulse_file)

# store results to the three-level controls dict
pi_half_pulse_controls[
    optimization_scheme
] = three_level_controls_from_two_level_optimized_result(results.best_output)

# plot controls
fig = plt.figure()
combined_controls = {**results.best_output}
plot_controls(fig, combined_controls)
fig.suptitle("Beam-Splitter pulse", y=0.95)
plt.show()

# M pulse
pi_duration = 9 * s  # duration of the M pulse
pi_target_operator = np.array([[0.0, -1j], [-1j, 0.0]], dtype=complex)
if not read_flag:
    print("**Optimizing Mirror pulse**")
    start_time = time.time()
    # the optimization routine
    results = run_optimization(pi_duration, pi_target_operator)
    save_var(Optimized_M_pulse_file, results)
    print("run (in min):", (time.time() - start_time) / 60.0)
else:
    results = load_var(Optimized_M_pulse_file)

# store results to the three-level controls dict
pi_pulse_controls[
    optimization_scheme
] = three_level_controls_from_two_level_optimized_result(results.best_output)

# plot controls
combined_controls = {**results.best_output}
fig = plt.figure()
plot_controls(fig, combined_controls)
fig.suptitle("Mirror pulse", y=0.95)
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: './resources/boosting-signal-to-noise-by-10x-in-cold-atom-sensors-using-robust-control/AtomSensing_Optimized_BS_pulse'

In [ ]:
#  generate pulses from Open Controls and add to the list of pulse controls
scheme = "primitive"

print(scheme, " Mirror pulse created")
pi_pulse = new_primitive_control(
    rabi_rotation=np.pi, azimuthal_angle=0, maximum_rabi_rate=omega_eff
)
# store the result in the control dict
pi_pulse_controls[scheme] = three_level_controls_from_Open_pulse(pi_pulse)

print(scheme, " Beam-Splitter pulse created")
pi_half_pulse = new_primitive_control(
    rabi_rotation=np.pi / 2, azimuthal_angle=0, maximum_rabi_rate=omega_eff
)
# store the result in the control dict
pi_half_pulse_controls[scheme] = three_level_controls_from_Open_pulse(pi_half_pulse)

In [ ]:
# set the three-level unitary target operators for the BS and M pulses
pi_half_target_operator = np.array(
    [
        [np.sqrt(2) / 2, 1j * np.sqrt(2) / 2, 0.0],
        [1j * np.sqrt(2) / 2, np.sqrt(2) / 2, 0.0],
        [0.0, 0.0, 1.0],
    ],
    dtype=complex,
)

pi_target_operator = np.array(
    [[0.0, 1j, 0.0], [1j, 0.0, 0.0], [0.0, 0.0, 1.0]], dtype=complex
)


In [ ]:
# quasi-static scan coefficients
betas = np.linspace(-0.4, 0.4, 100)
del_pz_coefficients = np.zeros_like(betas)

# pulse schemes
schemes = ["Q-CTRL", "primitive"]
colors = [QCTRL_STYLE_COLORS[0], QCTRL_STYLE_COLORS[1]]

fig, axs = plt.subplots(1, 2, figsize=(15, 5))
fig.suptitle("Robustness to fluctuations in laser amplitude", y=1.1)

# BS pulse quasi_static_scan
ax = axs[0]
ax.set_xlabel(r"$\beta$")
ax.set_ylabel("Infidelity")
ax.set_title("Beam-Splitter pulse")
ax.set_ylim(0, 0.7)

for idx, scheme in enumerate(schemes):
    controls = pi_half_pulse_controls[scheme]
    infidelities = pulse_1D_quasi_static_scan(
        controls, pi_half_target_operator, betas, del_pz_coefficients
    )
    ax.plot(betas, infidelities, label=scheme, color=colors[idx])

# M pulse quasi_static_scan
ax = axs[1]
ax.set_xlabel(r"$\beta$")
ax.set_ylabel("Infidelity")
ax.set_title("Mirror pulse")
ax.set_ylim(0, 0.7)

for idx, scheme in enumerate(schemes):
    controls = pi_pulse_controls[scheme]
    infidelities = pulse_1D_quasi_static_scan(
        controls, pi_target_operator, betas, del_pz_coefficients
    )
    ax.plot(betas, infidelities, label=scheme, color=colors[idx])

hs, ls = axs[0].get_legend_handles_labels()
fig.legend(handles=hs, labels=ls, loc="center", bbox_to_anchor=(0.5, 1.0), ncol=2)

plt.show()


In [ ]:
del_pzs = (h_bar * k_avr) * np.linspace(-1.2, 1.2, 100)
betas = np.zeros_like(del_pzs)

fig, axs = plt.subplots(1, 2, figsize=(15, 5))
fig.suptitle("Robustness to variations in atom momentum", y=1.1)

# BS pulse quasi_static_scan
ax = axs[0]
ax.set_xlabel(r"$\partial p_z/(\hbar k)$")
ax.set_ylabel("Infidelity")
ax.set_title("Beam-Splitter pulse")
ax.set_ylim(0, 0.2)
for idx, scheme in enumerate(schemes):
    controls = pi_half_pulse_controls[scheme]
    infidelities = pulse_1D_quasi_static_scan(
        controls, pi_half_target_operator, betas, del_pzs
    )
    ax.plot(del_pzs / (h_bar * k_avr), infidelities, label=scheme, color=colors[idx])

# M pulse quasi_static_scan
ax = axs[1]
ax.set_xlabel(r"$\partial p_z/(\hbar k)$")
ax.set_ylabel("Infidelity")
ax.set_title("Mirror pulse")
ax.set_ylim(0, 0.2)
for idx, scheme in enumerate(schemes):
    controls = pi_pulse_controls[scheme]
    infidelities = pulse_1D_quasi_static_scan(
        controls, pi_target_operator, betas, del_pzs
    )
    plt.plot(del_pzs / (h_bar * k_avr), infidelities, label=scheme, color=colors[idx])


hs, ls = axs[0].get_legend_handles_labels()
fig.legend(handles=hs, labels=ls, loc="center", bbox_to_anchor=(0.5, 1.0), ncol=2)

plt.show()


In [ ]:
# momentum noise coefficients
del_pzs = h_bar * k_avr * np.linspace(-2, 2, 40)
# laser noise coefficients beta
betas = np.linspace(-0.6, 0.6, 40)

# set up for 2D plot
gs = gridspec.GridSpec(1, 2)
fig = plt.figure(figsize=(15, 5))
fig.suptitle(
    "Robustness correlated variations in atom momentum and laser amplitude", y=1
)

scheme = schemes[0]
for k, scheme in enumerate(schemes):
    # quasistatic scan for the Mirror pulses
    (X, Y, Z) = pulse_2D_quasi_static_scan(
        pi_pulse_controls[scheme], pi_target_operator, betas, del_pzs
    )
    X = X / (h_bar * k_avr)

    # plot 2D contours
    ax = fig.add_subplot(gs[k])
    ax.set_title(scheme + " Mirror pulse")
    ax.set_ylabel(r"$\beta$")
    ax.set_xlabel(r"$\partial p_z/(\hbar k)$")
    contours = ax.contour(X, Y, Z, levels=[0.01, 0.1], colors=QCTRL_STYLE_COLORS[0])
    ax.clabel(contours, inline=True)
    cmap_reversed = plt.cm.get_cmap("gray").reversed()

    plt.imshow(
        Z,
        extent=[np.min(X), np.max(X), np.min(Y), np.max(Y)],
        origin="lower",
        cmap=cmap_reversed,
        alpha=0.8,
        aspect=np.abs((np.min(X) - np.max(X)) / (np.min(Y) - np.max(Y))),
        interpolation="bicubic",
        vmin=0,
        vmax=1,
    )
    cbar = plt.colorbar()
    cbar.set_label("Infidelity")

plt.show()


In [ ]:
ensemble_size = 2000  # the number of times we sample from the noise processes

# laser fluctuations, beta noise distribution
beta_sigma = 0.35

# set beta ensemble for each pulse
beta_ensembles = {}
for scheme in schemes:
    beta_ensembles[scheme] = []
    for _ in ["BS", "M", "BS"]:
        betas_for_pulse = np.random.normal(0, beta_sigma, ensemble_size)
        betas_for_pulse = np.clip(betas_for_pulse, -1, None)
        beta_ensembles[scheme].append(betas_for_pulse)

# thermal momentum noise
del_pz_sigma = 0.2 * h_bar * k_avr  # momentum width
pz0 = np.random.normal(0, del_pz_sigma, ensemble_size)

# compute ensemble of detunings
small_deltas = -2 * pz0 * k_avr / m_Rb
big_deltas = big_delta + pz0 * k_avr / m_Rb


In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 5))
fig.suptitle("High noise environment processes", y=1)

# plot laser noise
ax = axs[0]
ax.set_xlabel(r"$\beta$")
ax.set_ylabel("Probability density")
ax.set_title("Laser pulse amplitude distribution")
count, bins, ignored = ax.hist(
    beta_ensembles["primitive"][0], 30, density=True, color=QCTRL_STYLE_COLORS[2]
)
amplitude_distribution = np.exp(-(bins ** 2) / (2 * beta_sigma ** 2)) / (
    beta_sigma * np.sqrt(2 * np.pi)
)
ax.plot(bins, amplitude_distribution, "gray")

# plot momentum distibution
ax = axs[1]
ax.set_xlabel(r"$\partial p_z/\hbar k$")
ax.set_ylabel("Probability density")
ax.set_title("Thermal momentum distribution")

count, bins, ignored = plt.hist(
    pz0 / (h_bar * k_avr), 30, density=True, color=QCTRL_STYLE_COLORS[1]
)
thermal_distribution = (h_bar * k_avr / del_pz_sigma / np.sqrt(2 * np.pi)) * np.exp(
    -(bins ** 2) / (2 * (del_pz_sigma / (h_bar * k_avr)) ** 2)
)
plt.plot(bins, thermal_distribution, "gray")

plt.show()


In [ ]:
# the phase offset on the last BS pulse
phase_offsets = np.linspace(0, 6 * np.pi, 50)
# the initial state of the atom
initial_state = np.array([1, 0, 0])

signal_data = {}
if not read_flag:
    # the phase offset on the last BS pulse
    phase_offsets = np.linspace(0, 6 * np.pi, 50)
    # the initial state of the atom
    initial_state = np.array([1, 0, 0])

    start_time = time.time()
    for idx, scheme in enumerate(schemes):
        # evolve unitary though Mach–Zehnder sequence
        U_total = identity3
        # BS pulse
        U_BS1 = calculate_unitary(
            pi_half_pulse_controls[scheme],
            0,
            beta=beta_ensembles[scheme][0],
            big_delta=big_deltas,
            small_delta=small_deltas,
        )

        # M pulse
        U_M = calculate_unitary(
            pi_pulse_controls[scheme],
            0,
            beta=beta_ensembles[scheme][1],
            big_delta=big_deltas,
            small_delta=small_deltas,
        )

        # BS pulse
        U_BS2 = calculate_unitary(
            pi_half_pulse_controls[scheme],
            phase_offsets[:, None],
            beta=beta_ensembles[scheme][2][None, :],
            big_delta=big_deltas[None, :],
            small_delta=small_deltas[None, :],
        )

        # total unitary
        U_total = (U_BS1 @ U_M @ U_BS2).squeeze()

        # final |0> state population
        signal_ensemble = np.abs(U_total.dot(initial_state)[..., 0]) ** 2

        signal_data[scheme] = {
            "phase_offsets": phase_offsets,
            "signal_average": np.average(signal_ensemble, axis=1),
            "standard_error": np.std(signal_ensemble, axis=1) / np.sqrt(ensemble_size),
        }

    save_var(Fringe_signals_file, signal_data)
    end_time = time.time()

    print("run time: ", (end_time - start_time) / 60, "min")

else:
    signal_data = load_var(Fringe_signals_file)


In [ ]:
# plotting the fringes
fig, ax = plt.subplots(figsize=(15, 5))
fig.suptitle(
    r"Mach–Zehnder interferometer: fringe contrast comparison between primitive and optimized pulses",
)
plt.xlabel("Phase offset $\phi/\pi$")
plt.ylabel("Fringe signal")

# plot pz distibution
for idx, scheme in enumerate(schemes):
    x = signal_data[scheme]["phase_offsets"] / np.pi
    y = signal_data[scheme]["signal_average"]
    y_std = signal_data[scheme]["standard_error"]
    color = QCTRL_STYLE_COLORS[idx]
    plt.plot(x, y, "-", color=color, label=scheme)
    ax.fill_between(
        x,
        y - y_std,
        y + y_std,
        hatch="||",
        facecolor="none",
        edgecolor=color,
        linewidth=0.0,
        label=scheme + r" $\pm\sigma$",
    )

ax.set_xlim([0, 6])
ax.legend()

plt.show()
